In [7]:
import requests
from bs4 import BeautifulSoup
import re
import PIL.Image
import pytesseract
import numpy as np
import pandas as pd
domains = ['peersociallending.com', 'malta-tours.de', 'wiseclerk.com']

ModuleNotFoundError: No module named 'pytesseract'

In [8]:
emails = []
for domain in domains:
    try:
        url = f"https://{domain}/impressum"
        page = requests.get(url)
        soup = BeautifulSoup(page.content, "html.parser")
 
        # Search for email in text
        text = soup.get_text()
        email = re.search(r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+", text)
        if email:
            emails.append(email.group())
            continue

SyntaxError: unexpected EOF while parsing (3101579802.py, line 13)

In [9]:
 # Search for email in images
for img in soup.find_all("img"):
    src = img.get("src")
    if src.startswith("http"):
        image_content = requests.get(src).content
    else:
        base_url = f"https://{domain}"
        image_content = requests.get(base_url + src).content

    image = PIL.Image.open(BytesIO(image_content))
    text = pytesseract.image_to_string(image)

    email = re.search(r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+", text)
    if email:
        emails.append(email.group())
        break
 
    except:
        pass


SyntaxError: invalid syntax (48457329.py, line 18)

In [10]:
emails_padded = emails + [np.nan] * (len(domains) - len(emails))
df = pd.DataFrame({'Domain': domains, 'Email': emails_padded})
print(df)
# Output to Excel 
writer = pd.ExcelWriter('emails.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='Emails', index=False)
print("Results written to emails.xlsx")

NameError: name 'emails' is not defined

In [11]:
import requests
from bs4 import BeautifulSoup
import re
from PIL import Image
import pytesseract
import pandas as pd
from io import BytesIO

# List of domains to scrape
domains = [
    "domain1.com",
    "domain2.com",
    # Add more domains here
]

results = []

for domain in domains:
    result = {"Domain": domain, "Email": None}

    # Step 1: Extract email from "Impressum" page
    impressum_url = f"https://{domain}/impressum"
    response = requests.get(impressum_url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        email_elements = soup.find_all(string=re.compile(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'))
        
        if email_elements:
            result["Email"] = email_elements[0]
    
    # Step 2: Extract email from images using OCR
    if result["Email"] is None:
        image_url = f"https://{domain}/contact_image.png"  # Change to the actual image URL
        image_response = requests.get(image_url)

        if image_response.status_code == 200:
            image = Image.open(BytesIO(image_response.content))
            extracted_text = pytesseract.image_to_string(image)
            email_match = re.search(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', extracted_text)

            if email_match:
                result["Email"] = email_match.group()
    
    results.append(result)

# Convert results to a DataFrame and save as Excel
df = pd.DataFrame(results)
df.to_excel("scraped_emails.xlsx", index=False)


ModuleNotFoundError: No module named 'pytesseract'